# Video Game purchase generator

In [ ]:
!pip install pandas
!pip install faker
!pip install elasticsearch

In [4]:
from elasticsearch import Elasticsearch, helpers
import random
import uuid
from datetime import datetime
import faker

fake = faker.Faker()
es = Elasticsearch("http://localhost:9200")

def generate_bulk_actions(index, docs):
    for doc in docs:
        yield {
            "_index": index,
            "_id": doc.get("id", str(uuid.uuid4())),
            "_source": doc
        }

# Create games
games = []
genres = ["RPG", "Shooter", "Puzzle", "Platformer", "Strategy"]
for i in range(30):
    games.append({
        "id": f"game_{i}",
        "title": fake.catch_phrase(),
        "release_date": fake.date_between(start_date="-5y", end_date="today").isoformat(),
        "price": round(random.uniform(9.99, 69.99), 2),
        "genre": random.choice(genres)
    })

# Create users
users = []
regions = ["NA", "EU", "ASIA", "LATAM"]
for i in range(100):
    users.append({
        "id": f"user_{i}",
        "username": fake.user_name(),
        "signup_date": fake.date_between(start_date="-3y", end_date="today").isoformat(),
        "region": random.choice(regions),
        "age": random.randint(13, 55)
    })

# Create purchases
purchases = []
for i in range(1000):
    user = random.choice(users)
    game = random.choice(games)
    purchases.append({
        "id": str(uuid.uuid4()),
        "user_id": user["id"],
        "game_id": game["id"],
        "price_paid": round(game["price"] * random.uniform(0.5, 1.0), 2),
        "purchase_date": fake.date_between(
            start_date=datetime.fromisoformat(user["signup_date"]).date(),
            end_date=datetime.today().date()
        ).isoformat()
    })

# Create reviews
reviews = []
for i in range(500):
    user = random.choice(users)
    game = random.choice(games)
    reviews.append({
        "id": str(uuid.uuid4()),
        "user_id": user["id"],
        "game_id": game["id"],
        "rating": random.randint(1, 5),
        "review": fake.sentence(nb_words=12),
        "review_date": fake.date_between(
            start_date=datetime.fromisoformat(user["signup_date"]).date(),
            end_date=datetime.today().date()
        ).isoformat()
    })

# Bulk index everything
actions = []
actions.extend(generate_bulk_actions("games", games))
actions.extend(generate_bulk_actions("users", users))
actions.extend(generate_bulk_actions("purchases", purchases))
actions.extend(generate_bulk_actions("reviews", reviews))

helpers.bulk(es, actions)
print("✅ Done indexing all documents.")

✅ Done indexing all documents.
